<a href="https://colab.research.google.com/github/aicreativeexplorer/YT-Automation/blob/main/YT_Video_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install -q torch==2.2.0+cu118 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q diffusers transformers accelerate safetensors
!pip install -q flask flask_cors


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.6/811.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 728.5/728.5 MB ? eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 788.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [18]:
!pip install -q flask-cors
import os
import gc
import math
import time
import uuid
import threading
import subprocess
from pathlib import Path

import torch
import imageio
import requests
from PIL import Image

from diffusers import StableVideoDiffusionPipeline, StableDiffusionXLPipeline
from flask import Flask, request, jsonify, send_file
from flask_cors import CORS

# -------------------------------------------------------------------
# CONFIG
# -------------------------------------------------------------------

# Where to store generated videos
GEN_DIR = Path("/content/generated_videos")
GEN_DIR.mkdir(parents=True, exist_ok=True)

# Hugging Face token (set it in Colab or hardcode if you want)
HF_TOKEN = os.environ.get("HF_TOKEN", "").strip()

# Model IDs
SVD_XL_ID   = "stabilityai/stable-video-diffusion-img2vid-xt"   # primary
SVD_BASE_ID = "stabilityai/stable-video-diffusion-img2vid"      # fallback
SDXL_ID     = "stabilityai/stable-diffusion-xl-base-1.0"        # text→image

# Video settings
TARGET_SECONDS = 10          # 10-second videos
FPS            = 6           # 6 fps => ~60 frames for 10s (SVD will clamp)
HEIGHT         = 576         # SVD default-friendly res
WIDTH          = 1024

# Global model cache
GLOBAL_SVD_PIPE   = None
GLOBAL_SVD_MODEL  = None     # which SVD is active (XL or BASE)
GLOBAL_SDXL_PIPE  = None

# Job registry
JOBS = {}  # job_id -> { status, output_path, error?, prompt, seed_url, model_used }
JOBS_LOCK = threading.Lock()

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("DEVICE =", DEVICE)


DEVICE = cpu


In [5]:
def load_sdxl():
    """Load SDXL once for text→image (realistic/photoreal hybrid)."""
    global GLOBAL_SDXL_PIPE

    if GLOBAL_SDXL_PIPE is not None:
        print("[SDXL] Reusing cached SDXL pipeline")
        return GLOBAL_SDXL_PIPE

    auth = HF_TOKEN if HF_TOKEN else True
    print("[SDXL] Loading SDXL base model on", DEVICE)

    pipe = StableDiffusionXLPipeline.from_pretrained(
        SDXL_ID,
        torch_dtype=torch.float16 if DEVICE == "cuda" else torch.float32,
        use_auth_token=auth,
    )

    if DEVICE == "cuda":
        pipe.to("cuda")
        pipe.enable_attention_slicing("max")
    else:
        pipe.to("cpu")

    GLOBAL_SDXL_PIPE = pipe
    print("[SDXL] Loaded")
    return pipe


def load_svd():
    """
    Load SVD-XL first, fallback to SVD-base if OOM or failure.
    Returns (pipe, model_name)
    """
    global GLOBAL_SVD_PIPE, GLOBAL_SVD_MODEL

    if GLOBAL_SVD_PIPE is not None:
        print(f"[SVD] Reusing cached SVD: {GLOBAL_SVD_MODEL}")
        return GLOBAL_SVD_PIPE, GLOBAL_SVD_MODEL

    auth = HF_TOKEN if HF_TOKEN else True

    def _load(model_id):
        print(f"[SVD] Loading {model_id} on {DEVICE}")
        pipe = StableVideoDiffusionPipeline.from_pretrained(
            model_id,
            torch_dtype=torch.float16 if DEVICE == "cuda" else torch.float32,
            use_auth_token=auth,
        )
        if DEVICE == "cuda":
            pipe.to("cuda")
            pipe.enable_model_cpu_offload()
        else:
            pipe.to("cpu")
        return pipe

    # Try XL first
    try:
        pipe = _load(SVD_XL_ID)
        GLOBAL_SVD_PIPE = pipe
        GLOBAL_SVD_MODEL = "SVD_XL"
        print("[SVD] Loaded SVD-XL")
        return pipe, GLOBAL_SVD_MODEL
    except Exception as e:
        print("[SVD] Failed loading SVD-XL:", repr(e))
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    # Fallback to base
    try:
        pipe = _load(SVD_BASE_ID)
        GLOBAL_SVD_PIPE = pipe
        GLOBAL_SVD_MODEL = "SVD_BASE"
        print("[SVD] Loaded SVD-BASE as fallback")
        return pipe, GLOBAL_SVD_MODEL
    except Exception as e:
        print("[SVD] Failed loading SVD-BASE too:", repr(e))
        raise RuntimeError("Could not load any SVD model")


In [6]:
def download_image(url: str, dest: Path) -> Path:
    """Download image from URL to dest."""
    resp = requests.get(url, timeout=30)
    resp.raise_for_status()
    dest.parent.mkdir(parents=True, exist_ok=True)
    with open(dest, "wb") as f:
        f.write(resp.content)
    return dest


def frames_to_video(frames, out_path: Path, fps: int = FPS):
    """Convert list of PIL images to mp4 with imageio-ffmpeg."""
    out_path = Path(out_path)
    out_path.parent.mkdir(parents=True, exist_ok=True)
    writer = imageio.get_writer(out_path, fps=fps, codec="libx264")
    for frame in frames:
        writer.append_data(imageio.asarray(frame))
    writer.close()
    return out_path


def gpu_keepalive_loop():
    """Tiny CUDA ops to keep GPU 'awake' and reduce deallocation."""
    if DEVICE != "cuda":
        print("[KEEPALIVE] Not on GPU, skipping")
        return
    print("[KEEPALIVE] GPU keepalive thread started")
    while True:
        try:
            x = torch.randn((128, 128), device="cuda")
            y = x * 1.0000001
            del x, y
            if torch.cuda.is_available():
                torch.cuda.synchronize()
        except Exception as e:
            print("[KEEPALIVE] Error:", repr(e))
        time.sleep(120)  # every 2 min


# Start keepalive in background
keepalive_thread = threading.Thread(target=gpu_keepalive_loop, daemon=True)
keepalive_thread.start()


[KEEPALIVE] Not on GPU, skipping


In [7]:
def generate_image_from_text(prompt: str) -> Image.Image:
    """Use SDXL to create a high-quality still frame."""
    pipe = load_sdxl()

    # Slightly photoreal / cinematic hybrid
    enhanced_prompt = (
        f"{prompt}, ultra high quality, cinematic lighting, photorealistic, "
        "sharp details, 4k, masterpiece"
    )

    with torch.inference_mode():
        img = pipe(
            enhanced_prompt,
            height=HEIGHT,
            width=WIDTH,
            num_inference_steps=30,
        ).images[0]
    return img


def generate_video_svd(job_id: str, prompt: str, seed_image: Image.Image, duration: int = TARGET_SECONDS) -> Path:
    """
    Use SVD (XL or base) to create a video from one seed image.
    """
    pipe, model_name = load_svd()

    # SVD expects PIL upscaled to correct res
    img = seed_image.convert("RGB")
    img = img.resize((WIDTH, HEIGHT), Image.LANCZOS)

    num_frames = min(25, duration * FPS)  # clamp frames
    cfg_scale  = 2.5

    print(f"[SVD] Running {model_name} for job {job_id}: {num_frames} frames, {duration}s")

    with torch.inference_mode():
        result = pipe(
            img,
            num_frames=num_frames,
            decode_chunk_size=8,
            motion_bucket_id=127,  # mid-motion
            fps=FPS,
            noise_aug_strength=0.02,
            guidance_scale=cfg_scale,
        )

    frames = result.frames[0]  # (num_frames, H, W, C)

    # Convert to PILs and video
    pil_frames = [Image.fromarray(frame) for frame in frames]
    out_path = GEN_DIR / f"{job_id}.mp4"
    frames_to_video(pil_frames, out_path, fps=FPS)

    return out_path, model_name


In [8]:
app = Flask(__name__)
CORS(app)


@app.route("/health", methods=["GET"])
def health():
    gpu = torch.cuda.is_available()
    return jsonify({"ok": True, "gpu": gpu})


@app.route("/api/generate", methods=["POST"])
def api_generate():
    """
    Synchronous GPU job:
    - If seed_url provided → download + image→video
    - Else → text→image→video using SDXL + SVD
    Contract:
    - Request JSON: { id, prompt, duration, mode, seed_url? }
    - Response JSON: { ok: true/false, jobId, error? }
    """
    global JOBS

    data = request.get_json(silent=True) or request.form.to_dict() or {}

    job_id = data.get("id") or f"job-{uuid.uuid4().hex[:8]}"
    prompt = data.get("prompt", "A cinematic landscape")
    duration = int(data.get("duration", TARGET_SECONDS))
    duration = max(2, min(duration, 12))  # clamp to 2–12 sec
    seed_url = data.get("seed_url")

    with JOBS_LOCK:
        JOBS[job_id] = {
            "status": "running",
            "prompt": prompt,
            "seed_url": seed_url,
            "output_path": None,
            "error": None,
            "model_used": None,
        }

    try:
        # 1) Get seed image: image URL or SDXL
        if seed_url:
            img_path = GEN_DIR / f"{job_id}_seed.jpg"
            print(f"[API] Downloading seed image for job {job_id}")
            download_image(seed_url, img_path)
            seed_img = Image.open(img_path).convert("RGB")
        else:
            print(f"[API] Generating seed image from text for job {job_id}")
            seed_img = generate_image_from_text(prompt)

        # 2) SVD: image→video
        video_path, model_name = generate_video_svd(job_id, prompt, seed_img, duration)

        with JOBS_LOCK:
            JOBS[job_id]["status"] = "done"
            JOBS[job_id]["output_path"] = str(video_path)
            JOBS[job_id]["model_used"] = model_name

        print(f"[API] Job {job_id} DONE -> {video_path}")
        return jsonify({"ok": True, "jobId": job_id})

    except Exception as e:
        print(f"[API] Job {job_id} ERROR:", repr(e))
        with JOBS_LOCK:
            JOBS[job_id]["status"] = "error"
            JOBS[job_id]["error"] = repr(e)
        return jsonify({"ok": False, "jobId": job_id, "error": repr(e)}), 500


@app.route("/api/output/<job_id>", methods=["GET"])
def api_output(job_id):
    """
    Serve the generated MP4 for a given job.
    """
    with JOBS_LOCK:
        job = JOBS.get(job_id)

    if not job:
        return jsonify({"error": "job_not_found"}), 404

    if job.get("status") != "done" or not job.get("output_path"):
        return jsonify({"error": "not_ready"}), 404

    p = Path(job["output_path"])
    if not p.exists():
        return jsonify({"error": "file_missing"}), 404

    return send_file(str(p), mimetype="video/mp4", as_attachment=False)


In [19]:
PORT = 5000

print(f"Starting Flask server on port {PORT} ...")
app.run(host="0.0.0.0", port=PORT)


Starting Flask server on port 5000 ...
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


In [10]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared


In [11]:
import subprocess, re, time

def start_cloudflared(port=5000):
    cmd = [
        "./cloudflared",         # ← FIXED
        "tunnel",
        "--url", f"http://localhost:{port}",
        "--no-autoupdate"
    ]

    print(f"[CF] Starting Cloudflared on port {port}…")

    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True
    )

    url = None
    while True:
        line = proc.stdout.readline()
        if not line:
            break
        print("[CF]", line.strip())

        if "trycloudflare.com" in line:
            m = re.search(r"(https://[a-zA-Z0-9\-]+\.trycloudflare\.com)", line)
            if m:
                url = m.group(1)
                print("\n[CF] Public URL:", url, "\n")
                return url


In [20]:
cf_url = start_cloudflared(5000)
cf_url



[CF] Starting Cloudflared on port 5000…
[CF] 2025-12-08T09:16:12Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
[CF] 2025-12-08T09:16:12Z INF Requesting new quick Tunnel on trycloudflare.com...
[CF] 2025-12-08T09:16:16Z INF +--------------------------------------------------------------------------------------------+
[CF] 2025-12-08T09:16:16Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
[CF] 2025-12-08T09:16

'https://mighty-motherboard-studios-magnetic.trycloudflare.com'

In [13]:
!npm install -g localtunnel


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸
changed 22 packages in 3s
⠸
⠸3 packages are looking for funding
⠸  run `npm fund` for details
⠸

In [14]:
def start_localtunnel(port=5000, subdomain=None):
    cmd = ["lt", "--port", str(port)]
    if subdomain:
        cmd += ["--subdomain", subdomain]

    print(f"[LT] Starting LocalTunnel on port {port}…")

    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True
    )

    url = None
    while True:
        line = proc.stdout.readline()
        if not line:
            break
        print("[LT]", line.strip())

        if "https://" in line and ".loca.lt" in line:
            m = re.search(r"(https://[a-zA-Z0-9\-]+\.loca\.lt)", line)
            if m:
                url = m.group(1)
                print("\n[LT] Public URL:", url, "\n")
                return url


In [15]:
lt_url = start_localtunnel(5000)
lt_url


[LT] Starting LocalTunnel on port 5000…
[LT] your url is: https://bitter-states-leave.loca.lt

[LT] Public URL: https://bitter-states-leave.loca.lt 



'https://bitter-states-leave.loca.lt'